# FLEX-clash: how to poison a model

In this notebook, we will show how easy is to manipulate the model of a client using `flexclash`. Particularly, we show the usage of the decorator `@model_poisoned`, that allow us to easily modify the model of a client inside `FlexPool`.

In our first example, we will create a federated setup using MNIST with flex primitives and then poison the model of a client.

In [ ]:
from flex.datasets import load
import tensorflow as tf

flex_dataset = load("federated_emnist", return_test=False, split="digits")

In [ ]:
from flex.pool import FlexPool
from flex.pool.primitives_tf import init_server_model_tf


# Defining the model
def get_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

flex_pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=init_server_model_tf, model=get_model())

clients = flex_pool.clients
server = flex_pool.servers
aggregator = flex_pool.aggregators

print(f"Number of nodes in the pool {len(flex_pool)}: {len(server)} server plus {len(clients)} clients. The server is also an aggregator")

In [ ]:
from flex.pool.primitives_tf import deploy_server_model_tf

#Select clients
clients_per_round=20
selected_clients_pool = flex_pool.clients.select(clients_per_round)

server.map(deploy_server_model_tf, selected_clients_pool)

In [ ]:
from flex.pool.primitives_tf import train_tf

selected_clients_pool.map(train_tf, batch_size=512, epochs=1, verbose=False)

Now, just before sending the client's weights to the aggregator/server, we will randomize the weights of one random client

In [ ]:
from flexclash.model import model_poisoner
from flex.model import FlexModel
import numpy as np

randomized_weights_client = selected_clients_pool.select(1)

@model_poisoner
def weight_randomizer(client_model: FlexModel):
    rand_weights = [np.random.randn(*w.shape) for w in client_model["model"].get_weights()]
    client_model["model"].set_weights(rand_weights)
    return client_model

randomized_weights_client.map(weight_randomizer)